In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

project_id = 'gcp-cset-projects'
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

import plotly.express as px
import re

# CSET Style Hex Codes
# https://docs.google.com/document/d/1WqyNjRQEJvkI71z0Fin005VoPVT6G4N1OABwWlHuLBI/
cset_color_sequence = ["#0B1F41", "#003DA6", "#7C336F", "#B53A6D", "#3C8786", 
                     "#7AC4A5", "#F17F4C", "#15AFD0", "#839DC5", "E5BF21", "#B42025"]
cset_color_sequence[:1]

Authenticated


['#0B1F41']

In [2]:
%load_ext google.colab.data_table

In [3]:
# get language distribution
def get_lang_row(sql):
  lang_df = client.query(sql).to_dataframe()
  #Get all observations
  df_n = lang_df['N_all'].iloc[0]
  # get the key languages
  lang = lang_df.loc[lang_df['lang'].str.lower().isin(['english', 'chinese', 'german', 'french', 'spanish', 'portuguese', 'japanese']),]
  # convert language names to lower case
  lang.loc[:,'lang'] = lang['lang'].str.lower()
  # transpose
  lang_T = lang[['lang','N']].set_index('lang').T
  # add total observation for all languages
  lang_T['all'] = df_n
  row = lang_T[['all', 'english', 'chinese', 'german', 'french', 'spanish', 'portuguese', 'japanese']]
  return row


In [4]:
#Table 1 [row1 original publications]
sql = """
/* get total number of records */
 select * from (select count(merged_id) as N_all from frontiers_forecasting.article_links_2020_10_19 where merged_id in 
 ( select merged_id from frontiers_forecasting.corpus_merged where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null)),
 /* get records aggregated by language */
 (select * from
(select count(*) as N, lang from
(
select  merged_id, lang  from  (select merged_id, orig_id FROM  frontiers_forecasting.article_links_2020_10_19 ) c inner join
(select  title_cld2_lid_first_result as lang, id from  frontiers_forecasting.all_metadata_with_cld2_lid) l ON
c.orig_id = l.id where merged_id in ( select  merged_id from frontiers_forecasting.corpus_merged where doctype !=  "Patent" or  doctype != "Dataset" or  doctype is  Null
)
) group by lang) order by N desc)
"""
# get row
orig_row =  get_lang_row(sql)
orig_row

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



lang,all,english,chinese,german,french,spanish,portuguese,japanese
N,382809666,307078880,72059348,7643730,6083243,5341298,3686271,12963762


In [5]:
#Table 1 [row2 distinct publications]
sql = """
/* get total number of records */
 select * from (select count(distinct merged_id) as N_all from frontiers_forecasting.article_links_2020_10_19 where merged_id in 
 ( select merged_id from frontiers_forecasting.corpus_merged where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null)),
 /* get records aggregated by language */
 (select * from
(select count(*) as N, lang from
(
select distinct  merged_id, lang  from  (select merged_id, orig_id FROM  frontiers_forecasting.article_links_2020_10_19 ) c inner join
(select distinct title_cld2_lid_first_result as lang, id from  frontiers_forecasting.all_metadata_with_cld2_lid) l ON
c.orig_id = l.id where merged_id in ( select  merged_id from frontiers_forecasting.corpus_merged where doctype !=  "Patent" or  doctype != "Dataset" or  doctype is  Null
)
) group by lang) order by N desc)
"""

# Run a Standard SQL query using the environment's default project
dedup_row = get_lang_row(sql)
dedup_row

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



lang,all,english,chinese,german,french,spanish,portuguese,japanese
N,237552566,159852515,33496342,5206235,4669705,4732274,2814711,12119934


In [6]:
#Table 1 [row3 distinct articles in citation graph]
sql = """
/* get total number of records */
 select * from (select count(distinct merged_id) as N_all from gcp_cset_links_v2.article_links_2020_10_19 where merged_id in 
 ( select merged_id from gcp_cset_links_v2.corpus_merged where  (((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null) 
 and (merged_id in  (select id  from frontiers_forecasting.mapped_references) 
or merged_id in ( select ref_id  from frontiers_forecasting.mapped_references)) and merged_id in (select merged_id FROM  frontiers_forecasting.article_links_2020_10_19 )

 )),
 /* get records aggregated by language */
 (select * from
(select count(*) as N, lang from
(
select distinct  merged_id, lang  from  (select merged_id, orig_id FROM  frontiers_forecasting.article_links_2020_10_19 ) c inner join
(select distinct title_cld2_lid_first_result as lang, id from  frontiers_forecasting.all_metadata_with_cld2_lid) l ON
c.orig_id = l.id where merged_id in ( select  merged_id from frontiers_forecasting.corpus_merged where 
(doctype !=  "Patent" or  doctype != "Dataset" or  doctype is  Null) and (merged_id in  (select id  from frontiers_forecasting.mapped_references) 
or merged_id in ( select ref_id  from frontiers_forecasting.mapped_references)) and merged_id in (select merged_id FROM  frontiers_forecasting.article_links_2020_10_19 )
)
) group by lang) order by N desc)
"""


# Run a Standard SQL query using the environment's default project
graph_row = get_lang_row(sql)
graph_row

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



lang,all,english,chinese,german,french,spanish,portuguese,japanese
N,108832972,93711121,14122056,1958734,1137090,1041838,789137,521047


In [7]:
# Table 1 [row4 core science]
sql = """
/* get total number of records */
select * from 
 (select count(*) as N_all from frontiers_forecasting.all_data_20200610_filt_no_ref_from_and_ref_to_or_after_2017_reweighted_clean_clusters_1615e4_lvl2),
 /* get records aggregated by language */
(
select * from
(select count(*) as N, lang from
(
select distinct merged_id, lang  from  (select merged_id, orig_id FROM  frontiers_forecasting.article_links_2020_10_19 ) c inner join
(select distinct title_cld2_lid_first_result as lang, id from  frontiers_forecasting.all_metadata_with_cld2_lid) l ON
c.orig_id = l.id where merged_id in ( select distinct article_id from frontiers_forecasting.all_data_20200610_filt_no_ref_from_and_ref_to_or_after_2017_reweighted_clean_clusters_1615e4_lvl2)
) group by lang) order by N desc  
)
"""
core_row = get_lang_row(sql)
core_row

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



lang,all,english,chinese,german,french,spanish,portuguese,japanese
N,54709892,51579088,4041727,581740,259827,223741,228161,74610


In [8]:
# Table 1 [row5 clustered papers]
sql = """
/* get total number of records */
select * from 
 (select count(*) as N_all from frontiers_forecasting.dc5_cluster_assignment_latest),
 /* get records aggregated by language */
(
select * from
(select count(*) as N, lang from
(
select distinct merged_id, lang  from  (select merged_id, orig_id FROM  frontiers_forecasting.article_links_2020_10_19 ) c inner join
(select distinct title_cld2_lid_first_result as lang, id from  frontiers_forecasting.all_metadata_with_cld2_lid) l ON
c.orig_id = l.id where merged_id in (select article_id FROM  frontiers_forecasting.dc5_cluster_assignment_latest)
) group by lang) order by N desc 
)
"""
cl_row = get_lang_row(sql)
cl_row

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



lang,all,english,chinese,german,french,spanish,portuguese,japanese
N,104865885,92013834,11362767,1891107,1085553,974124,759435,496591


In [9]:
# Merge rows in Table 1
Table_lang = orig_row.append(dedup_row).append(graph_row).append(core_row).append(cl_row)
Table_lang['row_name'] = ['Original', 'De-duplicated','Cit-graph', 'Core-science', 'Clustered']
Table = Table_lang.set_index(['row_name'])
for c in Table.columns:
  Table.loc[:,c] = Table.loc[:,c].map(lambda x: f'{x:,}')


Table

lang,all,english,chinese,german,french,spanish,portuguese,japanese
row_name,,,,,,,,
Original,"382,809,666","307,078,880","72,059,348","7,643,730","6,083,243","5,341,298","3,686,271","12,963,762"
De-duplicated,"237,552,566","159,852,515","33,496,342","5,206,235","4,669,705","4,732,274","2,814,711","12,119,934"
Cit-graph,"108,832,972","93,711,121","14,122,056","1,958,734","1,137,090","1,041,838","789,137","521,047"
Core-science,"54,709,892","51,579,088","4,041,727","581,740","259,827","223,741","228,161","74,610"
Clustered,"104,865,885","92,013,834","11,362,767","1,891,107","1,085,553","974,124","759,435","496,591"


In [10]:
# Table 2 Country of Affliation

In [11]:
# row 1 all  documents
sql = """
WITH
country_tab_0 as (
select distinct merged_id, IF(alpha_3 = 'USA', 1,0) as USA_affiliation,  IF(alpha_3 = 'JPN', 1,0) as Japan_affiliation,
  IF( shared_functions.IsEU(alpha_3), 1,0) as EU_affiliation,   IF(alpha_3 = 'CHN', 1,0) as China_affiliation
from
(
select merged_id, alpha_3 from frontiers_forecasting.paper_affiliations_merged where merged_id in 
 ( select merged_id from frontiers_forecasting.corpus_merged where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null))
),
/* Chinese */
chinese as (
select distinct  merged_id  from  (select merged_id, orig_id FROM  frontiers_forecasting.article_links_2020_10_19 ) c inner join
(select  id from  frontiers_forecasting.all_metadata_with_cld2_lid where lower( title_cld2_lid_first_result) = 'chinese' ) l ON
c.orig_id = l.id where merged_id in ( select  merged_id from frontiers_forecasting.corpus_merged where doctype !=  "Patent" or  doctype != "Dataset" or  doctype is  Null)
),
/* Add CH affiliation for Chinese papers from CNKI */
corp as (
select distinct orig_id, USA_affiliation, Japan_affiliation, EU_affiliation, China_affiliation from
/* Add China affiliation */
(select merged_id, USA_affiliation, Japan_affiliation, EU_affiliation,
IF( merged_id in (select merged_id from frontiers_forecasting.article_links_2020_10_19 where orig_id like '%CNKI%') and
merged_id in (select merged_id from chinese), 1, China_affiliation) as  China_affiliation
/* get paper affiliations / countries  and merge it with ID dictionart*/
from country_tab_0)   c inner join (select merged_id as id, orig_id from frontiers_forecasting.article_links_2020_10_19) l on c.merged_id = l.id where merged_id in ( select merged_id from frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null)),
/* miss country */ 
miss_c as (
select count(distinct orig_id) as N_miss_aff from (select merged_id as id, orig_id from frontiers_forecasting.article_links_2020_10_19
 where merged_id in ( select merged_id from frontiers_forecasting.corpus_merged where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null))
  l left join (select merged_id from frontiers_forecasting.paper_affiliations_merged) c on c.merged_id = l.id where merged_id is null
),
 /* sum by country */
sum_c as (
select  sum(USA_affiliation) as N_USA, sum(Japan_affiliation) as N_Japan, sum(EU_affiliation) as N_EU, sum(China_affiliation) as N_China from
corp),
/* merge sum columns in a tables */
sum_all as (select count(distinct orig_id) as N_all from frontiers_forecasting.article_links_2020_10_19 
where merged_id in ( select merged_id from frontiers_forecasting.corpus_merged where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null))
select * from sum_all, miss_c  , sum_c
"""
orig_c = client.query(sql).to_dataframe()
orig_c

,N_all,N_miss_aff,N_USA,N_Japan,N_EU,N_China
0,382809666,193696919,56727937,10951007,60336932,31427474


In [12]:
# row 2 distinct  documents
sql = """
WITH
country_tab_0 as (
select distinct merged_id, IF(alpha_3 = 'USA', 1,0) as USA_affiliation,  IF(alpha_3 = 'JPN', 1,0) as Japan_affiliation,
  IF( shared_functions.IsEU(alpha_3), 1,0) as EU_affiliation,   IF(alpha_3 = 'CHN', 1,0) as China_affiliation
from
(
select merged_id, alpha_3 from frontiers_forecasting.paper_affiliations_merged where merged_id in ( select merged_id from frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null))
),
/* Chinese */
chinese as (
select distinct  merged_id  from  (select merged_id, orig_id FROM  frontiers_forecasting.article_links_2020_10_19 ) c inner join
(select  id from  frontiers_forecasting.all_metadata_with_cld2_lid where lower( title_cld2_lid_first_result) = 'chinese' ) l ON
c.orig_id = l.id where merged_id in ( select  merged_id from frontiers_forecasting.corpus_merged where doctype !=  "Patent" or  doctype != "Dataset" or  doctype is  Null)
),
/* Add CH affiliation for Chinese papers from CNKI */
corp as (

select distinct merged_id, USA_affiliation, Japan_affiliation, EU_affiliation, China_affiliation from
/* Add China affiliation */
(select merged_id, USA_affiliation, Japan_affiliation, EU_affiliation,
IF( merged_id in (select merged_id from frontiers_forecasting.article_links_2020_10_19 where orig_id like '%CNKI%') and
merged_id in (select merged_id from chinese), 1, China_affiliation) as  China_affiliation
/* get paper affiliations / countries  and merge it with ID dictionart*/
from country_tab_0)   c inner join (select merged_id as id from frontiers_forecasting.article_links_2020_10_19) l on c.merged_id = l.id 
where merged_id in ( select merged_id from frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null)),
 /* miss country */ 
miss_c as (
select count(distinct id) as N_miss_aff from (select merged_id as id from frontiers_forecasting.article_links_2020_10_19 
where merged_id in ( select merged_id from frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null)) l left join
 (select merged_id from frontiers_forecasting.paper_affiliations_merged) c on c.merged_id = l.id where merged_id is null
),
 /* sum by country */
sum_c as (
select  sum(USA_affiliation) as N_USA, sum(Japan_affiliation) as N_Japan, sum(EU_affiliation) as N_EU, sum(China_affiliation) as N_China from
corp),
/* merge sum columns in a tables */
sum_all as (select count(distinct merged_id) as N_all from frontiers_forecasting.article_links_2020_10_19
 where merged_id in ( select merged_id from frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null))
select * from sum_all,miss_c, sum_c
"""
de_dup_c = client.query(sql).to_dataframe()
de_dup_c 


,N_all,N_miss_aff,N_USA,N_Japan,N_EU,N_China
0,237552566,157601555,23990998,4530766,24802856,12608567


In [13]:
# row 3 citation graph
sql = """
WITH
country_tab_0 as (
select distinct merged_id, IF(alpha_3 = 'USA', 1,0) as USA_affiliation,  IF(alpha_3 = 'JPN', 1,0) as Japan_affiliation,
  IF( shared_functions.IsEU(alpha_3), 1,0) as EU_affiliation,   IF(alpha_3 = 'CHN', 1,0) as China_affiliation
from
(
select merged_id, alpha_3 from frontiers_forecasting.paper_affiliations_merged where merged_id in ( select merged_id from frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null))
),
/* Chinese */
chinese as (
select distinct  merged_id  from  (select merged_id, orig_id FROM  frontiers_forecasting.article_links_2020_10_19 ) c inner join
(select  id from  frontiers_forecasting.all_metadata_with_cld2_lid where lower( title_cld2_lid_first_result) = 'chinese' ) l ON
c.orig_id = l.id where merged_id in ( select  merged_id from frontiers_forecasting.corpus_merged where doctype !=  "Patent" or  doctype != "Dataset" or  doctype is  Null)
),
/* Add CH affiliation for Chinese papers from CNKI */
corp as (
select distinct merged_id, USA_affiliation, Japan_affiliation, EU_affiliation, China_affiliation from
/* Add China affiliation */
(select merged_id, USA_affiliation, Japan_affiliation, EU_affiliation,
IF( merged_id in (select merged_id from frontiers_forecasting.article_links_2020_10_19 where orig_id like '%CNKI%') and
merged_id in (select merged_id from chinese), 1, China_affiliation) as  China_affiliation
/* get paper affiliations / countries  and merge it with ID dictionart*/
from country_tab_0)   c inner join (select merged_id as id from frontiers_forecasting.article_links_2020_10_19) l on c.merged_id = l.id where merged_id in  
 ((select id  from frontiers_forecasting.mapped_references) UNION ALL ( select ref_id  from frontiers_forecasting.mapped_references)) and merged_id in 
  ( select merged_id from frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null)),
 /* miss affiliation */
 miss_c as (
select count(distinct id) as N_miss_aff from (select merged_id as id from frontiers_forecasting.article_links_2020_10_19) l left join (select merged_id from frontiers_forecasting.paper_affiliations_merged)
 c on c.merged_id = l.id where merged_id is null and id in
 ((select id  from frontiers_forecasting.mapped_references) UNION ALL ( select ref_id  from frontiers_forecasting.mapped_references)) and id in 
  ( select merged_id from frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null)
),
 /* sum by country */
sum_c as (
select  sum(USA_affiliation) as N_USA, sum(Japan_affiliation) as N_Japan, sum(EU_affiliation) as N_EU, sum(China_affiliation) as N_China from
corp),
/* merge sum columns in a tables */
sum_all as (select count(distinct merged_id) as N_all from frontiers_forecasting.article_links_2020_10_19 where merged_id in  
 ((select id  from frontiers_forecasting.mapped_references) UNION ALL ( select ref_id  from frontiers_forecasting.mapped_references)) and merged_id in  
 ( select merged_id from frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null))
 select * from sum_all,  miss_c, sum_c
"""
cit_gr_c = client.query(sql).to_dataframe()
cit_gr_c

,N_all,N_miss_aff,N_USA,N_Japan,N_EU,N_China
0,108911690,44042249,19272866,3697680,20583180,10847490


In [14]:
# row 4 core science
sql = """
WITH
country_tab_0 as (
select distinct merged_id, IF(alpha_3 = 'USA', 1,0) as USA_affiliation,  IF(alpha_3 = 'JPN', 1,0) as Japan_affiliation,
  IF( shared_functions.IsEU(alpha_3), 1,0) as EU_affiliation,   IF(alpha_3 = 'CHN', 1,0) as China_affiliation
from
(
select merged_id, alpha_3 from frontiers_forecasting.paper_affiliations_merged
)
),
/* Chinese */
chinese as (
select distinct  merged_id  from  (select merged_id, orig_id FROM  frontiers_forecasting.article_links_2020_10_19 ) c inner join
(select  id from  frontiers_forecasting.all_metadata_with_cld2_lid where lower( title_cld2_lid_first_result) = 'chinese' ) l ON
c.orig_id = l.id where merged_id in ( select  merged_id from frontiers_forecasting.corpus_merged where doctype !=  "Patent" or  doctype != "Dataset" or  doctype is  Null)
),
/* Add CH affiliation for Chinese papers from CNKI */
corp as (
select distinct merged_id, USA_affiliation, Japan_affiliation, EU_affiliation, China_affiliation from
/* Add China affiliation */
(select merged_id, USA_affiliation, Japan_affiliation, EU_affiliation,
IF( merged_id in (select merged_id from frontiers_forecasting.article_links_2020_10_19 where orig_id like '%CNKI%') and
merged_id in (select merged_id from chinese), 1, China_affiliation) as  China_affiliation
/* get paper affiliations / countries  and merge it with ID dictionart*/
from country_tab_0)   c inner join (select article_id as id from frontiers_forecasting.all_data_20200610_filt_no_ref_from_and_ref_to_or_after_2017_reweighted_clean_clusters_1615e4_lvl2) l on c.merged_id = l.id),
/* get missing affiliations */
 miss_c as (
select count(distinct id) as N_miss_aff from (select article_id as id from frontiers_forecasting.all_data_20200610_filt_no_ref_from_and_ref_to_or_after_2017_reweighted_clean_clusters_1615e4_lvl2) l left join (select merged_id from frontiers_forecasting.paper_affiliations_merged) c on c.merged_id = l.id where merged_id is null 
),
/* get sum by country */
sum_c as (
select  sum(USA_affiliation) as N_USA, sum(Japan_affiliation) as N_Japan, sum(EU_affiliation) as N_EU, sum(China_affiliation) as N_China from
corp),
sum_all as (select count(distinct article_id) as N_all from (select article_id from frontiers_forecasting.all_data_20200610_filt_no_ref_from_and_ref_to_or_after_2017_reweighted_clean_clusters_1615e4_lvl2)
)
select * from  sum_all,  miss_c, sum_c
"""
cit_core_sc = client.query(sql).to_dataframe()
cit_core_sc

,N_all,N_miss_aff,N_USA,N_Japan,N_EU,N_China
0,54709892,9970731,14180334,2710648,15161915,6436374


In [15]:
# row 5 clustered articles 
sql = """
WITH
country_tab_0 as (
select distinct merged_id, IF(alpha_3 = 'USA', 1,0) as USA_affiliation,  IF(alpha_3 = 'JPN', 1,0) as Japan_affiliation,
  IF( shared_functions.IsEU(alpha_3), 1,0) as EU_affiliation,   IF(alpha_3 = 'CHN', 1,0) as China_affiliation
from
(
select merged_id, alpha_3 from frontiers_forecasting.paper_affiliations_merged
)
),
/* Chinese */
chinese as (
select distinct  merged_id  from  (select merged_id, orig_id FROM  frontiers_forecasting.article_links_2020_10_19 ) c inner join
(select  id from  frontiers_forecasting.all_metadata_with_cld2_lid where lower( title_cld2_lid_first_result) = 'chinese' ) l ON
c.orig_id = l.id where merged_id in ( select  merged_id from frontiers_forecasting.corpus_merged where doctype !=  "Patent" or  doctype != "Dataset" or  doctype is  Null)
),
/* Add CH affiliation for Chinese papers from CNKI */
corp as (
select distinct merged_id, USA_affiliation, Japan_affiliation, EU_affiliation, China_affiliation from
/* Add China affiliation */
(select merged_id, USA_affiliation, Japan_affiliation, EU_affiliation,
IF( merged_id in (select merged_id from frontiers_forecasting.article_links_2020_10_19 where orig_id like '%CNKI%') and
merged_id in (select merged_id from chinese), 1, China_affiliation) as  China_affiliation
/* get paper affiliations / countries  and merge it with ID dictionart*/
from country_tab_0)   c inner join (select article_id as id from frontiers_forecasting.dc5_cluster_assignment_latest) l on c.merged_id = l.id),
/* get missing affiliations */
 miss_c as (
select count(distinct id) as N_miss_aff from (select article_id as id  from frontiers_forecasting.dc5_cluster_assignment_latest) l left join
 (select merged_id from frontiers_forecasting.paper_affiliations_merged) c on c.merged_id = l.id where merged_id is null 
),  
/* sum by country */
sum_c as (
select  sum(USA_affiliation) as N_USA, sum(Japan_affiliation) as N_Japan, sum(EU_affiliation) as N_EU, sum(China_affiliation) as N_China from
corp),
sum_all as (select count(distinct article_id) as N_all from frontiers_forecasting.dc5_cluster_assignment_latest)
select * from  sum_all,  miss_c, sum_c
"""
cl_sc = client.query(sql).to_dataframe()



In [16]:
# Merge rows in Table 2
Table_lang = orig_c.append(de_dup_c).append(cit_gr_c).append(cit_core_sc).append(cl_sc)
Table_lang['row_name'] = ['Original', 'De-duplicated','Cit-graph', 'Core-science', 'Clustered']
Table_lang = Table_lang.set_index(['row_name'])
Table = Table_lang[['N_all',	'N_miss_aff','N_China',	'N_EU','N_Japan','N_USA']]

for c in Table.columns:
  Table.loc[:,c] = Table.loc[:,c].map(lambda x: f'{x:,}')


Table

,N_all,N_miss_aff,N_China,N_EU,N_Japan,N_USA
row_name,,,,,,
Original,"382,809,666","193,696,919","31,427,474","60,336,932","10,951,007","56,727,937"
De-duplicated,"237,552,566","157,601,555","12,608,567","24,802,856","4,530,766","23,990,998"
Cit-graph,"108,911,690","44,042,249","10,847,490","20,583,180","3,697,680","19,272,866"
Core-science,"54,709,892","9,970,731","6,436,374","15,161,915","2,710,648","14,180,334"
Clustered,"104,865,885","40,556,630","10,623,969","20,477,187","3,684,186","19,195,775"


TABLE 3 Funders Information

In [17]:
# row 1 all  documents
#orig doc
sql = """
WITH funders as (
SELECT a.*, b.unique_funder FROM `gcp-cset-projects.frontiers_forecasting.article_links_2020_10_19` a
LEFT JOIN frontiers_forecasting.full_funder_table_wPaperID b USING(merged_id) where merged_id in ( select merged_id from frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null)
),
transpose_fund as (
select distinct * except(unique_funder), 
IF(unique_funder = 'jap_sps', 1,0) as jap_sps,
IF(unique_funder = 'nih',1,0) as nih,
IF(unique_funder = 'erc',1,0) as erc,
IF(unique_funder = 'ec',1,0) as ec,
IF(unique_funder = 'nsf',1,0) as nsf,
IF(unique_funder = 'nnsf_china',1,0) as nnsf_china,
from funders
),
cl_fund as (
(select merged_id, orig_id from (select merged_id, orig_id FROM `gcp-cset-projects.frontiers_forecasting.article_links_2020_10_19` where merged_id in ( select merged_id from 
frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null)) p inner join (select merged_id as id from frontiers_forecasting.paper_fundorg_merged) f ON p.merged_id = f.id)
),
other_fund_tab as (
select count(distinct orig_id) as N_other from cl_fund left join  (select merged_id, 1 as dis_fund from frontiers_forecasting.full_funder_table_wPaperID) f on 
cl_fund.merged_id = f.merged_id where dis_fund is null and cl_fund.merged_id in ( select merged_id from frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null)
),
sum_fund as (
(select count(distinct orig_id) as N, sum(ec) as ec , sum(erc) as erc, sum(jap_sps) as jap_sps, sum(nih) as nih, sum(nnsf_china) as nnsf_china, sum(nsf) as nsf from transpose_fund)
),
no_funder_tab as (
select count(distinct orig_id) as No_funder from
(select merged_id, orig_id FROM `gcp-cset-projects.frontiers_forecasting.article_links_2020_10_19` where merged_id in ( select merged_id from frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null)) p left join (select merged_id as id, 1 as ind from gcp_cset_links_v2.paper_fundorg_merged) f
 ON p.merged_id = f.id where ind is null
)
select N, no_funder, ec, erc, jap_sps, nih, nnsf_china, nsf, N_other   from sum_fund, other_fund_tab, no_funder_tab
"""
orig_row = client.query(sql).to_dataframe()
orig_row

,N,no_funder,ec,erc,jap_sps,nih,nnsf_china,nsf,N_other
0,382809666,327989947,2302659,683584,2110848,7995585,11281762,2341988,32377565


In [18]:
# Dedup IDs and funders:
sql = """
WITH funders as (
SELECT distinct a.merged_id, b.unique_funder FROM `gcp-cset-projects.frontiers_forecasting.article_links_2020_10_19` a
LEFT JOIN frontiers_forecasting.full_funder_table_wPaperID b USING(merged_id) where merged_id in ( select merged_id from 
frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null)
),
transpose_fund as (
select distinct * except(unique_funder), 
IF(unique_funder = 'jap_sps', 1,0) as jap_sps,
IF(unique_funder = 'nih',1,0) as nih,
IF(unique_funder = 'erc',1,0) as erc,
IF(unique_funder = 'ec',1,0) as ec,
IF(unique_funder = 'nsf',1,0) as nsf,
IF(unique_funder = 'nnsf_china',1,0) as nnsf_china,
from funders
),
cl_fund as (
(select distinct merged_id  from (select merged_id, orig_id FROM `gcp-cset-projects.frontiers_forecasting.article_links_2020_10_19` where merged_id in ( select merged_id from 
frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null)) p inner join (select merged_id as id from
 frontiers_forecasting.paper_fundorg_merged) f ON p.merged_id = f.id)
),
other_fund_tab as (
select count(distinct merged_id) as N_other from cl_fund left join  (select merged_id as id, 1 as dis_fund from frontiers_forecasting.full_funder_table_wPaperID) f on cl_fund.merged_id = f.id  where cl_fund.merged_id in ( select merged_id from 
frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null) and
 dis_fund is null
),
sum_fund as (
(select count(distinct merged_id) as N, sum(ec) as ec , sum(erc) as erc, sum(jap_sps) as jap_sps, sum(nih) as nih, sum(nnsf_china) as nnsf_china, sum(nsf) as nsf from transpose_fund)
),
no_funder_tab as (
select count(distinct merged_id) as no_funder from
(select merged_id, orig_id FROM `gcp-cset-projects.frontiers_forecasting.article_links_2020_10_19` where merged_id in ( select merged_id from 
frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null)) p left join (select merged_id as id, 1 as ind from gcp_cset_links_v2.paper_fundorg_merged) f 
ON p.merged_id = f.id where ind is null
)
select N, no_funder, ec, erc, jap_sps, nih, nnsf_china, nsf, N_other   from sum_fund, other_fund_tab, no_funder_tab
"""
dedup_row = client.query(sql).to_dataframe()
dedup_row

,N,no_funder,ec,erc,jap_sps,nih,nnsf_china,nsf,N_other
0,237552566,219229222,741669,197204,689055,2924962,4165945,727657,10734979


In [19]:
# citation graph and funders
sql = """
WITH funders as (
SELECT distinct a.merged_id, b.unique_funder FROM `gcp-cset-projects.frontiers_forecasting.article_links_2020_10_19` a
LEFT JOIN frontiers_forecasting.full_funder_table_wPaperID b USING(merged_id) where merged_id in  
 ((select id  from frontiers_forecasting.mapped_references) UNION ALL ( select ref_id  from frontiers_forecasting.mapped_references))
),
transpose_fund as (
select distinct * except(unique_funder), 
IF(unique_funder = 'jap_sps', 1,0) as jap_sps,
IF(unique_funder = 'nih',1,0) as nih,
IF(unique_funder = 'erc',1,0) as erc,
IF(unique_funder = 'ec',1,0) as ec,
IF(unique_funder = 'nsf',1,0) as nsf,
IF(unique_funder = 'nnsf_china',1,0) as nnsf_china,
from funders where  merged_id in  
 ((select id  from frontiers_forecasting.mapped_references) UNION ALL ( select ref_id  from frontiers_forecasting.mapped_references)) and merged_id in 
  ( select merged_id from frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null)
),
cl_fund as (
(select distinct merged_id  from (select merged_id, orig_id FROM `gcp-cset-projects.frontiers_forecasting.article_links_2020_10_19`) p inner join (select merged_id as id from 
frontiers_forecasting.paper_fundorg_merged) f ON p.merged_id = f.id where merged_id in  
 ((select id  from frontiers_forecasting.mapped_references) UNION ALL ( select ref_id  from frontiers_forecasting.mapped_references)) and merged_id in 
  ( select merged_id from frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null))
),
other_fund_tab as (
select count(distinct merged_id) as N_other from cl_fund left join  (select merged_id as id, 1 as dis_fund from frontiers_forecasting.full_funder_table_wPaperID) f on cl_fund.merged_id = f.id 
where dis_fund is null and merged_id in  
 ((select id  from frontiers_forecasting.mapped_references) UNION ALL ( select ref_id  from frontiers_forecasting.mapped_references)) and merged_id in 
  ( select merged_id from frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null)
),
sum_fund as (
(select count(distinct merged_id) as N, sum(ec) as ec , sum(erc) as erc, sum(jap_sps) as jap_sps, sum(nih) as nih, sum(nnsf_china) as nnsf_china, sum(nsf) as nsf
 from transpose_fund)
),
no_funder_tab as (
select count(distinct merged_id) as No_funder from
(select merged_id, orig_id FROM `gcp-cset-projects.frontiers_forecasting.article_links_2020_10_19`) p left join (select merged_id as id, 1 as ind from gcp_cset_links_v2.paper_fundorg_merged) f
 ON p.merged_id = f.id where ind is null and  merged_id in  
 ((select id  from frontiers_forecasting.mapped_references) UNION ALL ( select ref_id  from frontiers_forecasting.mapped_references)) and merged_id in 
  ( select merged_id from frontiers_forecasting.corpus_merged
 where  ((doctype !=  "Patent") and (doctype != "Dataset")) or  doctype is Null)
)
select N, no_funder, ec, erc, jap_sps, nih, nnsf_china, nsf, N_other   from sum_fund, other_fund_tab, no_funder_tab
"""
graph_row = client.query(sql).to_dataframe()
graph_row

,N,no_funder,ec,erc,jap_sps,nih,nnsf_china,nsf,N_other
0,108911690,90867761,739357,196163,681897,2886553,4007241,724033,10539562


In [20]:
#core science  and funders
sql = """
WITH funders as (
SELECT distinct article_id as merged_id, b.unique_funder FROM frontiers_forecasting.all_data_20200610_filt_no_ref_from_and_ref_to_or_after_2017_reweighted_clean_clusters_1615e4_lvl2 a
LEFT JOIN frontiers_forecasting.full_funder_table_wPaperID b ON a.article_id = b.merged_id
),
transpose_fund as (
select distinct * except(unique_funder), 
IF(unique_funder = 'jap_sps', 1,0) as jap_sps,
IF(unique_funder = 'nih',1,0) as nih,
IF(unique_funder = 'erc',1,0) as erc,
IF(unique_funder = 'ec',1,0) as ec,
IF(unique_funder = 'nsf',1,0) as nsf,
IF(unique_funder = 'nnsf_china',1,0) as nnsf_china,
from funders
),
cl_fund as (
(select distinct merged_id  from (select article_id as id from frontiers_forecasting.all_data_20200610_filt_no_ref_from_and_ref_to_or_after_2017_reweighted_clean_clusters_1615e4_lvl2) p inner join (select merged_id from frontiers_forecasting.paper_fundorg_merged) f ON p.id = f.merged_id)
),
other_fund_tab as (
select count(distinct merged_id) as N_other from cl_fund left join  (select merged_id as id, 1 as dis_fund from frontiers_forecasting.full_funder_table_wPaperID) f on cl_fund.merged_id = f.id where dis_fund is null
),
sum_fund as (
(select count(distinct merged_id) as N, sum(ec) as ec , sum(erc) as erc, sum(jap_sps) as jap_sps, sum(nih) as nih, sum(nnsf_china) as nnsf_china, sum(nsf) as nsf from transpose_fund)
),
no_funder_tab as (
select count(distinct merged_id) as No_funder from
(select article_id as merged_id from frontiers_forecasting.all_data_20200610_filt_no_ref_from_and_ref_to_or_after_2017_reweighted_clean_clusters_1615e4_lvl2) p left join (select merged_id as id, 1 as ind from gcp_cset_links_v2.paper_fundorg_merged) f ON p.merged_id = f.id where ind is null
)
select N, no_funder, ec, erc, jap_sps, nih, nnsf_china, nsf, N_other   from sum_fund, other_fund_tab, no_funder_tab
"""
core_row = client.query(sql).to_dataframe()
core_row

,N,no_funder,ec,erc,jap_sps,nih,nnsf_china,nsf,N_other
0,54709892,39754602,643374,167116,583031,2534835,2612271,640194,8693716


In [21]:
#clustered funders
sql = """
WITH funders as (
SELECT distinct article_id as merged_id, b.unique_funder FROM frontiers_forecasting.dc5_cluster_assignment_latest a
LEFT JOIN frontiers_forecasting.full_funder_table_wPaperID b ON a.article_id = b.merged_id
),
transpose_fund as (
select distinct * except(unique_funder), 
IF(unique_funder = 'jap_sps', 1,0) as jap_sps,
IF(unique_funder = 'nih',1,0) as nih,
IF(unique_funder = 'erc',1,0) as erc,
IF(unique_funder = 'ec',1,0) as ec,
IF(unique_funder = 'nsf',1,0) as nsf,
IF(unique_funder = 'nnsf_china',1,0) as nnsf_china,
from funders
),
cl_fund as (
(select distinct merged_id  from (select article_id as id from frontiers_forecasting.dc5_cluster_assignment_latest) p inner join (select merged_id from frontiers_forecasting.paper_fundorg_merged)
 f ON p.id = f.merged_id)
),
other_fund_tab as (
select count(distinct merged_id) as N_other from cl_fund left join  (select merged_id as id, 1 as dis_fund from frontiers_forecasting.full_funder_table_wPaperID) f 
on cl_fund.merged_id = f.id where dis_fund is null
),
sum_fund as (
(select count(distinct merged_id) as N, sum(ec) as ec , sum(erc) as erc, sum(jap_sps) as jap_sps, sum(nih) as nih, sum(nnsf_china) as nnsf_china, sum(nsf) as nsf
from transpose_fund)
),
no_funder_tab as (
select count(distinct merged_id) as No_funder from
(select article_id as merged_id from frontiers_forecasting.dc5_cluster_assignment_latest) p left join (select merged_id as id, 1 as ind from gcp_cset_links_v2.paper_fundorg_merged) f 
ON p.merged_id = f.id where ind is null
)
select N, no_funder, ec, erc, jap_sps, nih, nnsf_china, nsf, N_other   from sum_fund, other_fund_tab, no_funder_tab
"""
cl_row = client.query(sql).to_dataframe()
cl_row

,N,no_funder,ec,erc,jap_sps,nih,nnsf_china,nsf,N_other
0,104865885,86856051,738616,195971,681177,2885273,3973858,723441,10519492


In [22]:
# Merge rows in Table 1
Table_lang = orig_row.append(dedup_row).append(graph_row).append(core_row).append(cl_row)
Table_lang['row_name'] = ['Original', 'De-duplicated','Cit-graph', 'Core-science', 'Clustered']
Table = Table_lang.set_index(['row_name'])

for c in Table.columns:
  Table.loc[:,c] = Table.loc[:,c].map(lambda x: f'{x:,}')


Table

,N,no_funder,ec,erc,jap_sps,nih,nnsf_china,nsf,N_other
row_name,,,,,,,,,
Original,"382,809,666","327,989,947","2,302,659","683,584","2,110,848","7,995,585","11,281,762","2,341,988","32,377,565"
De-duplicated,"237,552,566","219,229,222","741,669","197,204","689,055","2,924,962","4,165,945","727,657","10,734,979"
Cit-graph,"108,911,690","90,867,761","739,357","196,163","681,897","2,886,553","4,007,241","724,033","10,539,562"
Core-science,"54,709,892","39,754,602","643,374","167,116","583,031","2,534,835","2,612,271","640,194","8,693,716"
Clustered,"104,865,885","86,856,051","738,616","195,971","681,177","2,885,273","3,973,858","723,441","10,519,492"


In [23]:
# Table funders concetration
sql = """
WITH
nnsf_tab as (
select  "China" as country, "NNSF_China" as funder, 
avg(IF( nnsf_conc = 1 , x_growth ,null))  as share_high_g_for_funded_AI,
avg( IF(China_affiliation_share = 0, null, nnsf_china_share / China_affiliation_share )) as share_nat_ai_supported,
avg(IF(x_growth=1,China_affiliation_share,null)) as country_share_high_g,
avg(IF(x_growth=0,China_affiliation_share,null))  as country_share_low_g,
from frontiers_forecasting.funders_forecast_2020
),
nsf_tab as (
select  "USA" as country, "NSF" as funder,
avg(IF(nsf_conc=1,x_growth,null))  as share_high_g_for_funded_AI,
avg( IF(USA_affiliation_share  = 0, null, nsf_share / USA_affiliation_share ) )as share_nat_ai_supported,
avg(IF(x_growth=1,USA_affiliation_share,null)) as country_share_high_g,
avg(IF(x_growth=0,USA_affiliation_share,null))  as country_share_low_g,
from frontiers_forecasting.funders_forecast_2020
),
nih_tab as (
select  "USA" as country, "NIH" as funder,
avg(IF(nih_conc=1,x_growth,null))  as share_high_g_for_funded_AI,
avg( IF(USA_affiliation_share  = 0, null, nih_share / USA_affiliation_share )) as share_nat_ai_supported,
avg(IF(x_growth=1,USA_affiliation_share,null)) as country_share_high_g,
avg(IF(x_growth=0,USA_affiliation_share,null))  as country_share_low_g,
from frontiers_forecasting.funders_forecast_2020
),
erc_tab as (
select  "EU" as country, "ERC" as funder,
avg(IF(erc_conc=1,x_growth,null))  as share_high_g_for_funded_AI,
avg( IF(EU_affiliation_share  = 0, null, erc_share / EU_affiliation_share )) as share_nat_ai_supported,
avg(IF(x_growth=1,EU_affiliation_share,null)) as country_share_high_g,
avg(IF(x_growth=0,EU_affiliation_share,null))  as country_share_low_g,
from frontiers_forecasting.funders_forecast_2020
),
ec_tab as (
select  "EU" as country, "EC" as funder,
avg(IF(ec_conc=1,x_growth,null))  as share_high_g_for_funded_AI,
avg( IF(EU_affiliation_share  = 0, null, ec_share / EU_affiliation_share )) as share_nat_ai_supported,
avg(IF(x_growth=1,EU_affiliation_share,null)) as country_share_high_g,
avg(IF(x_growth=0,EU_affiliation_share,null))  as country_share_low_g,
from frontiers_forecasting.funders_forecast_2020
),
eu_tab as (
select  "EU" as country, "EU" as funder,
avg(IF(eu_conc=1,x_growth,null))  as share_high_g_for_funded_AI,
avg( IF(EU_affiliation_share  = 0, null, eu_share / EU_affiliation_share )) as share_nat_ai_supported,
avg(IF(x_growth=1,EU_affiliation_share,null)) as country_share_high_g,
avg(IF(x_growth=0,EU_affiliation_share,null))  as country_share_low_g,
from frontiers_forecasting.funders_forecast_2020
),
jap_sps_tab as (
select  "Japan" as country, "JAP_SPS" as funder,
avg(IF(jap_sps_conc=1,x_growth,null))  as share_high_g_for_funded_AI,
avg(IF(Japan_affiliation_share  = 0, null, jap_sps_share/ Japan_affiliation_share)) as share_nat_ai_supported,
avg(IF(x_growth=1,Japan_affiliation_share,null)) as country_share_high_g, 
avg(IF(x_growth=0,Japan_affiliation_share,null)) as country_share_low_g, 
from frontiers_forecasting.funders_forecast_2020
)
select *, 1 - share_high_g_for_funded_AI as share_low_g_for_funded_AI from
(
select * from nnsf_tab UNION ALL (select * from nsf_tab) UNION ALL (select * from nih_tab)
UNION ALL (select * from erc_tab) UNION ALL (select * from ec_tab) UNION ALL (select * from eu_tab)   UNION ALL (select * from jap_sps_tab)
) order by funder
"""
targ_tab = client.query(sql).to_dataframe()
targ_tab[['country', 'funder', 'share_high_g_for_funded_AI', 'share_low_g_for_funded_AI'	,'share_nat_ai_supported'	,'country_share_high_g'	,
         'country_share_low_g']]

,country,funder,share_high_g_for_funded_AI,share_low_g_for_funded_AI,share_nat_ai_supported,country_share_high_g,country_share_low_g
0,EU,EC,0.219269,0.780731,0.092679,0.171911,0.212986
1,EU,ERC,0.362126,0.637874,0.037990,0.171911,0.212986
2,EU,EU,0.232558,0.767442,0.083302,0.171911,0.212986
3,Japan,JAP_SPS,0.266667,0.733333,0.367466,0.033082,0.045547
4,USA,NIH,0.300000,0.700000,0.081216,0.242482,0.183099
5,China,NNSF_China,0.306667,0.693333,0.701880,0.314375,0.286247
6,USA,NSF,0.350000,0.650000,0.120018,0.242482,0.183099


In [24]:
sql = """
select sum(ROUND( EU_affiliation_share * NP)) as eu,    sum(ROUND( USA_affiliation_share * NP)) as us,
 sum(ROUND( China_affiliation_share * NP)) as ch,    sum(ROUND( Japan_affiliation_share * NP)) as jap, sum(NP) as world
from frontiers_forecasting.funders_forecast_2020
"""
ai_pub_nat = client.query(sql).to_dataframe()
ai_pub_nat

,eu,us,ch,jap,world
0,187624.0,183355.0,355068.0,40430.0,1032850


In [25]:
sql = """
select sum(ROUND( EU_affiliation_share * NP)) as eu,    sum(ROUND( USA_affiliation_share * NP)) as us,
 sum(ROUND( China_affiliation_share * NP)) as ch,    sum(ROUND( Japan_affiliation_share * NP)) as jap, sum(NP) as world
from frontiers_forecasting.funders_forecast_2020 where x_growth = 1
"""
ai_pub_x_gr = client.query(sql).to_dataframe()
ai_pub_x_gr

,eu,us,ch,jap,world
0,53867.0,72259.0,127378.0,11132.0,344022


In [26]:
sql = """
select
ROUND(sum(ec_share * NP)) as EC,
ROUND(sum(erc_share * NP)) as ERC,
ROUND(sum(jap_sps_share * NP)) as JPS,
ROUND(sum(nih_share * NP)) as NIH,
ROUND(sum(nnsf_china_share * NP)) as NNSF,
ROUND(sum(nsf_share * NP)) as NSF
from frontiers_forecasting.funders_forecast_2020
"""
funder_ai = client.query(sql).to_dataframe()
funder_ai

,EC,ERC,JPS,NIH,NNSF,NSF
0,18731.0,7037.0,16139.0,18821.0,256526.0,22984.0


In [27]:
sql = """
select
ROUND(sum(ec_share * NP)) as EC,
ROUND(sum(erc_share * NP)) as ERC,
ROUND(sum(jap_sps_share * NP)) as JPS,
ROUND(sum(nih_share * NP)) as NIH,
ROUND(sum(nnsf_china_share * NP)) as NNSF,
ROUND(sum(nsf_share * NP)) as NSF,
from frontiers_forecasting.funders_forecast_2020 where x_growth = 1
"""
funder_ai_x_growht = client.query(sql).to_dataframe()
funder_ai_x_growht

,EC,ERC,JPS,NIH,NNSF,NSF
0,4702.0,2999.0,4954.0,8223.0,102326.0,9327.0


In [28]:
fund = funder_ai.append(funder_ai_x_growht).T
fund.columns = ['Funded', 'Funded XGRC']
fund['country'] = ['eu', 'eu', 'jap', 'us', 'ch', 'us']
pub = ai_pub_nat.append(ai_pub_x_gr).T
pub.columns = ['Published', 'Published XGRC']

In [29]:
tab_dis = pub.merge(fund, how = 'left', left_index=True, right_on = 'country')
tab_dis['pub_x_share'] = tab_dis['Published XGRC']/tab_dis['Published']
tab_dis['fund_x_share'] = tab_dis['Funded XGRC']/tab_dis['Funded']
tab_dis['funder'] = tab_dis.index
tab_dis[['country', 'Published', 'Published XGRC', 'pub_x_share','funder', 'Funded', 'Funded XGRC', 'fund_x_share']].sort_values(by=['country','funder' ])

,country,Published,Published XGRC,pub_x_share,funder,Funded,Funded XGRC,fund_x_share
NNSF,ch,355068.0,127378.0,0.358743,NNSF,256526.0,102326.0,0.398891
EC,eu,187624.0,53867.0,0.287101,EC,18731.0,4702.0,0.251028
ERC,eu,187624.0,53867.0,0.287101,ERC,7037.0,2999.0,0.426176
JPS,jap,40430.0,11132.0,0.275340,JPS,16139.0,4954.0,0.306958
NIH,us,183355.0,72259.0,0.394093,NIH,18821.0,8223.0,0.436906
NSF,us,183355.0,72259.0,0.394093,NSF,22984.0,9327.0,0.405804
NaN,world,1032850.0,344022.0,0.333080,NaN,NaN,NaN,NaN


In [30]:
# Average
sql = """
select XG/NP as xg_share from
(
select sum(NP* x_growth) as XG, sum(NP) as NP from frontiers_forecasting.funders_forecast_2020
) 
"""
x_growht_sh_ave = client.query(sql).to_dataframe()
x_growht_sh_ave

,xg_share
0,0.33308
